### 好像需要 brightness tempurature?
因為要建超簡易模型的話好像需要...  
現在沒什麼概念但是這種轉換應該是個比較通用的需求  
總之又滾來做了   

湯底 = oldData/v2/4_brightnessTemp_ratio/wrr/py_mom0_pipeline_oldData_brightnessTemp.ipynb  

但加入的材料(cube)是不一樣的  
這邊加入的是精選所有最小的 beam 數據 (沒有smooth過)  
順便期望修復一下之前沒解決的問題（即轉換後數值爆炸）

0814  
好像解決的，但是原理沒有深究

In [ ]:
from casatasks import importfits, immath, imhead, exportfits
import shutil
import os
import numpy as np

In [ ]:
# costants
c = 3e8 # m/s
k_B = 1.380649e-23 # J/K


'''
因為這裡都是讀入處理良好的 cube (裁切並且寫入正確的 restfreq)
所以可以直接從 header 裡取出 f_r
'''
commonPath = '/Users/aqing/Documents/1004/Circinus_galaxy/Ratio-Map/tinyBeam/1_cube_product/cube'
# (mole, beam(arcsec))
mole_info = [#('CO-1-0', 1.64), #這個忘改標頭了幹
        #('CO-2-1', 0.39, '6'), #這個太巨了
        ('CO-3-2', 0.29, '7'),
        ('HCO+-1-0', 2.89, '3'),
        ('HCO+-2-1', 1.32, '5'),
        ('HCO+-3-2', 1.47, '6'),
        ('HCO+-4-3', 0.29, '7'),
        ]

'\n因為這裡都是讀入處理良好的 cube (裁切並且寫入正確的 restfreq)\n所以可以直接從 header 裡取出 f_r\n'

In [ ]:
for mole, beam, band in mole_info:
    print(f"Start changing the unit of flux density for {mole}'s cube.")
    pathIN = f'{commonPath}_band{band}_{mole}_beam{beam}_cropped.fits'
    fitsOUT = f'cube_band{band}_{mole}_beam{beam}_cropped_unit-K.fits' # 最後輸出的 cube
    importfits(fitsimage=pathIN, imagename='casaIN.im', overwrite=True)

    f0 = imhead(imagename='casaIN.im', mode="get", hdkey="restfreq")['value'] # Hz
    
    sr = 1.1331 * ((beam/3600)*(np.pi/180))**2
    factor = ((c**2)/(2*k_B*sr*(f0**2))) * 1e-26 # 這個 1e-26 沒有仔細理解過

    immath(imagename='casaIN.im', expr=f'{factor} * IM0', outfile='K.im')
    imhead(imagename='K.im', mode='put', hdkey='bunit', hdvalue='K') # 標示改個

    exportfits(imagename='K.im', fitsimage=fitsOUT, overwrite=True)

    shutil.rmtree('casaIN.im')
    shutil.rmtree('K.im') # 雖然 overwrite=True，還是移一下

print('FIN')


Start changing the unit of flux density for CO-3-2's cube.
Start changing the unit of flux density for HCO+-1-0's cube.
Start changing the unit of flux density for HCO+-2-1's cube.
Start changing the unit of flux density for HCO+-3-2's cube.
Start changing the unit of flux density for HCO+-4-3's cube.
FIN
